# Processing Big Data - Data Ingestion
© Explore Data Science Academy

## Honour Code
I {OLUSOJI, ONIGBINDE}, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the [EDSA honour code](https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).
    Non-compliance with the honour code constitutes a material breach of contract.



## Context 

To work constructively with any dataset, one needs to create an ingestion profile to make sure that the data at the source can be readily consumed. For this section of the predict, as the Data Engineer in the team, you will be required to design and implement the ingestion process. For the purposes of the project the AWS cloud storage service, namely, the S3 bucket service will act as your data source. All the data required can be found [here](https://processing-big-data-predict-stocks-data.s3.eu-west-1.amazonaws.com/stocks.zip).

<div align="center" style="width: 600px; font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/data_engineering/transform/predict/DataIngestion.jpg"
     alt="Data Ingestion"
     style="float: center; padding-bottom=0.5em"
     width=40%/>
     <p><em>Figure 1. Data Ingestion</em></p>
</div>

Your manager, Gnissecorp Atadgib, knowing very well that you've recently completed your Data Engineering qualification, asks you to make use of Apache Spark for the ingestion as well as the rest of the project. His rationale being, that stock market data is generated every day and is quite time-sensitive and would require scalability when deploying to a production environment. 

## Dataset - US Nasdaq




<div align="center" style="width: 600px; font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/data_engineering/transform/predict/Nasdaq.png"
     alt="Nasdaq"
     style="float: center; padding-bottom=0.5em"
     width=50%/>
     <p><em>Figure 2. Nasdaq</em></p>
</div>

The data that you will be working with is a historical snapshot of market data taken from the Nasdaq electronic market. This dataset contains historical daily prices for all tickers currently trading on Nasdaq. The up-to-date list can be found on their [website](https://www.nasdaq.com/)


The provided data contains price data dating back from 02 January 1962 up until 01 April 2020. The data found in the S3 bucket has been stored in the following structure:

```
     stocks/<Year>/<Month>/<Day>/stocks.csv
```
Each CSV file for every trading day contains the following details:
- **Date** - specifies trading date
- **Open** - opening price
- **High** - maximum price during the day
- **Low** - minimum price during the day
- **Close** - close price adjusted for splits
- **Adj Close** - close price adjusted for both dividends and splits
- **Volume** - the number of shares that changed hands during a given day

## Basic initialisation
To get you started, let's import some basic Python libraries as well as Spark modules and functions.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *

Remember that we need a `SparkContext` and `SparkSession` to interface with Spark.
We will mostly be using the `SparkContext` to interact with RDDs and the `SparkSession` to interface with Python objects.

> ℹ️ **Instructions** ℹ️
>
>Initialise a new **Spark Context** and **Session** that you will use to interface with Spark.

In [2]:
#TODO: Write your code here
sc = SparkContext()
spark = SparkSession(sc)

In [3]:
spark

## Investigate dataset schema
At this point, it is enough to read in a single file to ascertain the data structure. You will be required to use the information obtained from the small subset to create a data schema. This data schema will be used when reading the entire dataset using Spark.

> ℹ️ **Instructions** ℹ️
>
>Make use of Pandas to read in a single file and investigate the plausible data types to be used when creating a Spark data schema. 
>
>*You may use as many coding cells as necessary.*

In [4]:
#INSERTED
pd_df = pd.read_csv('stocks/1962/01/02/stocks.csv')
pd_df[:5]

,Date,Open,High,Low,Close,Adj Close,Volume,stock
0,1962-01-02,6.532155,6.556185,6.532155,6.532155,1.536658,55900.0,AA
1,1962-01-02,6.125844,6.160982,6.125844,6.125844,1.414651,59700.0,ARNC
2,1962-01-02,0.837449,0.837449,0.823045,0.823045,0.145748,352200.0,BA
3,1962-01-02,1.604167,1.619792,1.588542,1.604167,0.136957,163200.0,CAT
4,1962-01-02,0.000000,3.296131,3.244048,3.296131,0.051993,105600.0,CVX


In [5]:
#INSERTED
pd_df.describe()

,Open,High,Low,Close,Adj Close,Volume
count,20.000000,20.000000,20.000000,20.000000,2.000000e+01,2.000000e+01
mean,1.202568,18.146229,17.829750,17.967168,6.248174e+00,4.490900e+05
std,2.457011,58.093799,56.989958,57.542460,2.597106e+01,7.027279e+05
min,0.000000,0.096026,0.092908,0.092908,6.281419e-07,0.000000e+00
25%,0.000000,0.640337,0.623523,0.627279,1.185386e-02,4.445000e+04
50%,0.000000,2.457961,2.416295,2.450149,1.413525e-01,1.344000e+05
75%,0.772764,8.051069,7.933194,7.933194,8.265629e-01,4.920000e+05
max,7.713333,263.125000,258.125000,260.625000,1.165582e+02,2.480300e+06


In [6]:
#INSERTED
df = spark.read.csv('stocks/1962/01/02/stocks.csv', header=True)
df.show(5)

+----------+------------------+------------------+------------------+------------------+-------------------+--------+-----+
|      Date|              Open|              High|               Low|             Close|          Adj Close|  Volume|stock|
+----------+------------------+------------------+------------------+------------------+-------------------+--------+-----+
|1962-01-02|6.5321550369262695| 6.556184768676758|6.5321550369262695|6.5321550369262695| 1.5366575717926023| 55900.0|   AA|
|1962-01-02| 6.125843524932861| 6.160982131958008| 6.125843524932861| 6.125843524932861| 1.4146506786346436| 59700.0| ARNC|
|1962-01-02|0.8374485373497009|0.8374485373497009|0.8230452537536621|0.8230452537536621| 0.1457476019859314|352200.0|   BA|
|1962-01-02|1.6041666269302368|1.6197916269302368|1.5885416269302368|1.6041666269302368|0.13695742189884186|163200.0|  CAT|
|1962-01-02|               0.0|3.2961308956146236|3.2440476417541504|3.2961308956146236| 0.0519925132393837|105600.0|  CVX|
+-------

In [7]:
#TODO: Write your code here
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Adj Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- stock: string (nullable = true)



## Read CSV files

When working with big data, it is often not tenable to keep processing an entire data batch when you are in the process of development - this can be quite time-consuming. If the data is uniform, it is sufficient to work with a smaller subset to create basic functionality. Your manager has identified the year **1962** to perform the initial testing for data ingestion. 

> ℹ️ **Instructions** ℹ️
>
>Read in the data for **1962** using a data schema that purely uses string data types. You will be required to convert to the appropriate data types at a later stage.
>
>*You may use as many coding cells as necessary.*

In [8]:
#INSERTED
import os

stocks_data = []
parent_dir = r'stocks/1962/'
for subdir, dirs, files in os.walk(parent_dir):
    for file in files:
        if file.endswith(".csv"):
            stocks_data.append(os.path.join(subdir, file))           
        else:
            continue

In [9]:
#INSERTED
stocks_data

['stocks/1962/01\\02\\stocks.csv',
 'stocks/1962/01\\03\\stocks.csv',
 'stocks/1962/01\\04\\stocks.csv',
 'stocks/1962/01\\05\\stocks.csv',
 'stocks/1962/01\\08\\stocks.csv',
 'stocks/1962/01\\09\\stocks.csv',
 'stocks/1962/01\\10\\stocks.csv',
 'stocks/1962/01\\11\\stocks.csv',
 'stocks/1962/01\\12\\stocks.csv',
 'stocks/1962/01\\15\\stocks.csv',
 'stocks/1962/01\\16\\stocks.csv',
 'stocks/1962/01\\17\\stocks.csv',
 'stocks/1962/01\\18\\stocks.csv',
 'stocks/1962/01\\19\\stocks.csv',
 'stocks/1962/01\\22\\stocks.csv',
 'stocks/1962/01\\23\\stocks.csv',
 'stocks/1962/01\\24\\stocks.csv',
 'stocks/1962/01\\25\\stocks.csv',
 'stocks/1962/01\\26\\stocks.csv',
 'stocks/1962/01\\29\\stocks.csv',
 'stocks/1962/01\\30\\stocks.csv',
 'stocks/1962/01\\31\\stocks.csv',
 'stocks/1962/02\\01\\stocks.csv',
 'stocks/1962/02\\02\\stocks.csv',
 'stocks/1962/02\\05\\stocks.csv',
 'stocks/1962/02\\06\\stocks.csv',
 'stocks/1962/02\\07\\stocks.csv',
 'stocks/1962/02\\08\\stocks.csv',
 'stocks/1962/02\\09

In [10]:
#TODO: Write your code here
df_1 = spark.read \
     .format('csv') \
     .option('header',True) \
     .load(stocks_data)

In [11]:
#INSERTED
df_1.show(5180)

+----------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-----+
|      Date|                Open|                High|                 Low|               Close|           Adj Close|    Volume|stock|
+----------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-----+
|1962-02-19|   5.839290142059326|   5.907374858856201|   5.839290142059326|   5.863319873809815|  1.3863292932510376|   29900.0|   AA|
|1962-02-19|   5.481634140014648|  5.5284857749938965|   5.481634140014648|   5.516772747039795|  1.2804527282714844|   32000.0| ARNC|
|1962-02-19|  0.9074074029922484|  0.9156378507614136|  0.8991769552230835|   0.903292179107666|  0.1614154428243637|  619400.0|   BA|
|1962-02-19|  1.6770833730697632|  1.6927083730697632|  1.6614583730697632|  1.6770833730697632|  0.1440587043762207|  170400.0|  CAT|
|1962-02-19|                 0.0|  3.5788691043853764| 

In [12]:
#INSERTED
df_1.describe().toPandas()

,summary,Date,Open,High,Low,Close,Adj Close,Volume,stock
0,count,5106,5106,5106,5084,5106,5106,5085,5106
1,mean,None,1.0904873526012002,16.757624946793637,15.728619917198033,16.64199179044607,5.986642513535306,540930.2458210423,None
2,stddev,None,2.3644535253049095,53.91407348193544,51.322922898144,53.546771399008016,24.64637054715319,864596.2440525509,None
3,min,1962-01-02,0.0,0.0,-0.040435523172560696F44,0.053624976426362984,-0.04100876298375362F22,0.0,AA
4,max,1962-12-31,7.7133331298828125,9.984375,9.96875,9.984375,7.029978519312864e-07,998400.0,XOM


## Update column names
To make the data easier to work with, you will need to make a few changes:
1. Column headers should all be in lowercase; and
2. Whitespaces should be replaced with underscores.


> ℹ️ **Instructions** ℹ️
>
>Make sure that the column headers are all in lowercase and that any whitespaces are replaced with underscores.
>
>*You may use as many coding cells as necessary.*

In [13]:
#TODO: Write your code here
for column in df_1.columns:
    df_1 = df_1.withColumnRenamed(column, '_'.join(column.split()).lower())

In [14]:
#INSERTED
df_1.columns

['date', 'open', 'high', 'low', 'close', 'adj_close', 'volume', 'stock']

## Null Values
Null values often represent missing pieces of data. It is always good to know where your null values lie - so you can quickly identify and remedy any issues stemming from these.

> ℹ️ **Instructions** ℹ️
>
>Write code to count the number of null values found in each column.
>
>*You may use as many coding cells as necessary.*

In [15]:
#TODO: Write your code here
df_1_nVal = df_1.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df_1.columns])

In [16]:
#INSERTED
df_1_nVal.show()

+----+----+----+---+-----+---------+------+-----+
|date|open|high|low|close|adj_close|volume|stock|
+----+----+----+---+-----+---------+------+-----+
|   0|   0|   0| 22|    0|        0|    21|    0|
+----+----+----+---+-----+---------+------+-----+



## Data type conversion - The final data schema

Now that we have identified the number of missing values in the data set, we'll move on to convert our data schema to the required data types. 

> ℹ️ **Instructions** ℹ️
>
>Use typecasting to convert the string data types in your current data schema to more appropriate data types.
>
>*You may use as many coding cells as necessary.*

In [17]:
#TODO: Write your code here
schema = StructType([StructField('date', StringType(), True),
                     StructField('open', FloatType(), True),
                     StructField('high', FloatType(), True),
                     StructField('low', FloatType(), True),
                     StructField('close', FloatType(), True),
                     StructField('adj_close', FloatType(), True),
                     StructField('Volume', FloatType(), True),
                     StructField('stock', StringType(), True)])


df_2 = spark.read.csv(stocks_data, header=True, schema=schema, multiLine=True)

In [18]:
#INSERTED
df_2.show()

+----------+-----------+-----------+----------+-----------+------------+---------+-----+
|      date|       open|       high|       low|      close|   adj_close|   Volume|stock|
+----------+-----------+-----------+----------+-----------+------------+---------+-----+
|1962-02-19|    5.83929|   5.907375|   5.83929|    5.86332|   1.3863293|  29900.0|   AA|
|1962-02-19|   5.481634|   5.528486|  5.481634|  5.5167727|   1.2804527|  32000.0| ARNC|
|1962-02-19|  0.9074074| 0.91563785|0.89917696|  0.9032922|  0.16141544| 619400.0|   BA|
|1962-02-19|  1.6770834|  1.6927084| 1.6614584|  1.6770834|   0.1440587| 170400.0|  CAT|
|1962-02-19|        0.0|   3.578869|      20.0|   3.549107| 0.056501225| 273600.0|  CVX|
|1962-02-19|0.099767394|0.099767394|0.09820853| 0.09820853| 0.037543412| 817400.0|  DIS|
|1962-02-19|        0.0|    29.9375|     29.75|    29.9375|  0.49964145|   1600.0|  DTE|
|1962-02-19|        0.0|   9.921875|  9.890625|   9.921875|  0.22499175|   8800.0|   ED|
|1962-02-19|        0

In [19]:
#INSERTED (convert date from string format to time-stamp format)
df_2 = df_2.withColumn('date', F.to_timestamp(F.col('date')))

In [20]:
#INSERTED
df_2.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- open: float (nullable = true)
 |-- high: float (nullable = true)
 |-- low: float (nullable = true)
 |-- close: float (nullable = true)
 |-- adj_close: float (nullable = true)
 |-- Volume: float (nullable = true)
 |-- stock: string (nullable = true)



## Consolidate missing values
We have to check if the data type conversion above was done correctly.
If the casting was not successful, a null value gets inserted into the dataframe. You can thus check for successful conversion by determining if any null values are included in the resulting dataframe.


> ℹ️ **Instructions** ℹ️
>
>Write code to compare the number of invalid entries (nulls) pre-conversion and post-conversion.
>
>*You may use as many coding cells as necessary.*

In [21]:
#TODO: Write your code here
df_2_nVal = df_2.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df_2.columns])

df_1_nVal.show()
df_2_nVal.show()

+----+----+----+---+-----+---------+------+-----+
|date|open|high|low|close|adj_close|volume|stock|
+----+----+----+---+-----+---------+------+-----+
|   0|   0|   0| 22|    0|        0|    21|    0|
+----+----+----+---+-----+---------+------+-----+

+----+----+----+---+-----+---------+------+-----+
|date|open|high|low|close|adj_close|Volume|stock|
+----+----+----+---+-----+---------+------+-----+
|   0|   0|   0| 42|    0|       21|    21|    0|
+----+----+----+---+-----+---------+------+-----+



Here you should be able to see if any of your casts went wrong. 
Do not attempt to correct any missing values at this point. This will be dealt with in later sections of the predict.

## Generate parquet files
When writing in Spark, we typically use parquet format. This format allows parallel writing using Spark's optimisation while maintaining other useful things like metadata.

When writing, it is good to make sure that the data is sufficiently partitioned. 

Generally, data should be partitioned with one partition for every 200MB of data, but this also depends on the size of your cluster and executors. 


### Check the size of the dataframe before partitioning

In [22]:
from pyspark.serializers import PickleSerializer, AutoBatchedSerializer

In [23]:
rdd = df.rdd._reserialize(AutoBatchedSerializer(PickleSerializer()))
obj = rdd.ctx._jvm.org.apache.spark.mllib.api.python.SerDe.pythonToJava(rdd._jrdd, True)
size = sc._jvm.org.apache.spark.util.SizeEstimator.estimate(obj)
size_MB = size/1000000
partitions = max(int(size_MB/200), 2)
print(f'The dataframe is {size_MB} MB')
print(f'partitions is {partitions}')

The dataframe is 11.79932 MB
partitions is 2


In [24]:
rdd.getNumPartitions()

1

### Write parquet files to the local directory
> ℹ️ **Instructions** ℹ️
>
> Use the **coalesce** function and the number of **partitions** derived above to write parquet files to your local directory 
>
>*You may use as many coding cells as necessary.*

In [25]:
#TODO: Write your code here
df_2.coalesce(partitions).write.parquet('transformed_data')

AnalysisException: path file:/C:/Users/hp/Desktop/pbd/transformed_data already exists.

In [ ]:
spark.stop()